In [1]:
import os
print(os.getcwd())


C:\Users\syksh


In [2]:
import os
os.chdir('C:/Users/syksh/Documents/Instacart')  # Change this to your data folder path


In [3]:
print(os.getcwd())

C:\Users\syksh\Documents\Instacart


In [4]:
import pandas as pd

# Load each dataset
orders = pd.read_csv('orders.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')
products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')  # Optional
departments = pd.read_csv('departments.csv')  # Optional

# Check if the files are loaded properly by displaying the first few rows of one file
print(orders.head())  # This will display the first 5 rows of the 'orders' DataFrame


   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0  


In [5]:
import os
# List all files in the current directory
files = os.listdir()
print(files)


['aisles.csv', 'departments.csv', 'orders.csv', 'order_products__prior.csv', 'order_products__train.csv', 'products.csv', 'sample_submission.csv']


In [6]:
# Combine prior and train datasets into one
order_products = pd.concat([order_products_prior, order_products_train])

# Preview the first few rows of the combined dataset
print(order_products.head())


   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0


In [7]:
# Merge orders with order_products on 'order_id'
orders_combined = pd.merge(order_products, orders, on='order_id', how='inner')

# Preview the first few rows of the merged dataset
print(orders_combined.head())


   order_id  product_id  add_to_cart_order  reordered  user_id eval_set  \
0         2       33120                  1          1   202279    prior   
1         2       28985                  2          1   202279    prior   
2         2        9327                  3          0   202279    prior   
3         2       45918                  4          1   202279    prior   
4         2       30035                  5          0   202279    prior   

   order_number  order_dow  order_hour_of_day  days_since_prior_order  
0             3          5                  9                     8.0  
1             3          5                  9                     8.0  
2             3          5                  9                     8.0  
3             3          5                  9                     8.0  
4             3          5                  9                     8.0  


In [8]:
# Merge with products to include product names
orders_with_products = pd.merge(orders_combined, products, on='product_id', how='inner')

# Preview the first few rows
print(orders_with_products.head())


   order_id  product_id  add_to_cart_order  reordered  user_id eval_set  \
0         2       33120                  1          1   202279    prior   
1         2       28985                  2          1   202279    prior   
2         2        9327                  3          0   202279    prior   
3         2       45918                  4          1   202279    prior   
4         2       30035                  5          0   202279    prior   

   order_number  order_dow  order_hour_of_day  days_since_prior_order  \
0             3          5                  9                     8.0   
1             3          5                  9                     8.0   
2             3          5                  9                     8.0   
3             3          5                  9                     8.0   
4             3          5                  9                     8.0   

            product_name  aisle_id  department_id  
0     Organic Egg Whites        86             16  
1  Mic

In [9]:
# Merge with aisles and departments (optional, for additional detail)
orders_full = pd.merge(orders_with_products, aisles, on='aisle_id', how='left')
orders_full = pd.merge(orders_full, departments, on='department_id', how='left')

# Preview the final combined dataset
print(orders_full.head())


   order_id  product_id  add_to_cart_order  reordered  user_id eval_set  \
0         2       33120                  1          1   202279    prior   
1         2       28985                  2          1   202279    prior   
2         2        9327                  3          0   202279    prior   
3         2       45918                  4          1   202279    prior   
4         2       30035                  5          0   202279    prior   

   order_number  order_dow  order_hour_of_day  days_since_prior_order  \
0             3          5                  9                     8.0   
1             3          5                  9                     8.0   
2             3          5                  9                     8.0   
3             3          5                  9                     8.0   
4             3          5                  9                     8.0   

            product_name  aisle_id  department_id               aisle  \
0     Organic Egg Whites        86   

In [10]:
# Check the shape of the final dataset
print(orders_full.shape)

# Display the first few rows
print(orders_full.head())


(33819106, 15)
   order_id  product_id  add_to_cart_order  reordered  user_id eval_set  \
0         2       33120                  1          1   202279    prior   
1         2       28985                  2          1   202279    prior   
2         2        9327                  3          0   202279    prior   
3         2       45918                  4          1   202279    prior   
4         2       30035                  5          0   202279    prior   

   order_number  order_dow  order_hour_of_day  days_since_prior_order  \
0             3          5                  9                     8.0   
1             3          5                  9                     8.0   
2             3          5                  9                     8.0   
3             3          5                  9                     8.0   
4             3          5                  9                     8.0   

            product_name  aisle_id  department_id               aisle  \
0     Organic Egg Whit

In [12]:
# Recency: Find the most recent order for each customer
recency = orders_full.groupby('user_id')['days_since_prior_order'].min().reset_index()
recency.columns = ['user_id', 'recency']

# Frequency: Count the total number of orders per customer
frequency = orders_full.groupby('user_id')['order_id'].nunique().reset_index()
frequency.columns = ['user_id', 'frequency']

# Merge Recency and Frequency into one DataFrame
rfm = pd.merge(recency, frequency, on='user_id')

# Preview the RFM DataFrame
print(rfm.head())


   user_id  recency  frequency
0        1      0.0         11
1        2      3.0         15
2        3      7.0         12
3        4      0.0          5
4        5      6.0          5


In [17]:
# Calculate Monetary: Total number of products purchased by each customer
monetary = orders_full.groupby('user_id')['product_id'].count().reset_index()
monetary.columns = ['user_id', 'monetary']

# Merge the Monetary DataFrame with the existing RFM DataFrame
rfm = pd.merge(rfm, monetary, on='user_id')


In [18]:
# Assign scores based on quartiles
rfm['recency_score'] = pd.qcut(rfm['recency'], q=4, labels=[4, 3, 2, 1])  # Higher score for more recent purchases
rfm['frequency_score'] = pd.qcut(rfm['frequency'], q=4, labels=[1, 2, 3, 4])  # Higher score for more frequent purchases
rfm['monetary_score'] = pd.qcut(rfm['monetary'], q=4, labels=[1, 2, 3, 4])  # Higher score for higher spending


In [19]:
# Combine RFM scores into a single RFM score
rfm['RFM_score'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str)


In [20]:
# Define customer segments based on RFM_score
def segment_customer(rfm_score):
    if rfm_score == '444':
        return 'High Value'
    elif rfm_score in ['344', '434', '443']:
        return 'Loyal'
    elif rfm_score in ['144', '214', '124']:
        return 'At Risk'
    else:
        return 'Other'

# Apply the function to create the 'segment' column
rfm['segment'] = rfm['RFM_score'].apply(segment_customer)
                    

In [21]:
# Specify the path to save the CSV file
file_path = r"C:\Users\syksh\Documents\CustomerSegmentationProject\data\rfm_segments.csv"

# Save the DataFrame as a CSV file to the specified path
rfm.to_csv(file_path, index=False)


In [22]:
orders_with_segments = pd.merge(orders_full, rfm[['user_id', 'segment']], on='user_id', how='left')


In [23]:
# Group by segment and product_id, then count purchases
top_products_by_segment = orders_with_segments.groupby(['segment', 'product_id']).size().reset_index(name='purchase_count')


In [24]:
# Merge to add product names
top_products_by_segment = pd.merge(top_products_by_segment, products[['product_id', 'product_name']], on='product_id', how='left')


In [25]:
top_products_by_segment = top_products_by_segment.sort_values(['segment', 'purchase_count'], ascending=[True, False])


In [26]:
# Define the file path
top_products_file_path = r"C:\Users\syksh\Documents\CustomerSegmentationProject\data\top_products_by_segment.csv"

# Save to CSV
top_products_by_segment.to_csv(top_products_file_path, index=False)


In [27]:
# Group by order_dow and segment, then count orders
orders_by_day = orders_with_segments.groupby(['order_dow', 'segment']).size().reset_index(name='order_count')

# Define the file path for the CSV file
orders_by_day_file_path = r"C:\Users\syksh\Documents\CustomerSegmentationProject\data\orders_by_day.csv"

# Save the DataFrame to a CSV file
orders_by_day.to_csv(orders_by_day_file_path, index=False)


In [28]:
# Group by order_dow, order_hour_of_day, and segment, then count orders
orders_by_day_hour = orders_with_segments.groupby(['order_dow', 'order_hour_of_day', 'segment']).size().reset_index(name='order_count')

# Define the file path for the CSV file
orders_by_day_hour_file_path = r"C:\Users\syksh\Documents\CustomerSegmentationProject\data\orders_by_day_hour.csv"

# Save the DataFrame to a CSV file
orders_by_day_hour.to_csv(orders_by_day_hour_file_path, index=False)
